# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.firefox import GeckoDriverManager
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

/Users/shreyaraman/.pyenv/versions/3.10.3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(
/var/folders/71/p219_qns5mv0bqxkh541lpl40000gn/T/ipykernel_8530/1912161385.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

In [3]:
Select(driver.find_element(By.ID, "pht_status")).select_by_visible_text("Cosmetologists")

In [4]:
driver.find_element(By.ID, "pht_lnm").send_keys("Nguyen")

In [5]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't actual tabular data!

> You can use either Selenium by itself or Selenium+BeautifulSoup to scrape the results page. The choice is up to you!

### Loop through each result and print the entire row

Okay wait, maybe not, i's a heck of a lot of rows. Use `[:10]` to only do the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

> *Tip: If you're using Selenium, `By.TAG_NAME` is used if you don't have a class or ID. If you're using BeautifulSoup, just do your normal thing.*

In [6]:
rows= driver.find_elements(By.TAG_NAME, "tr")

for row in rows[:10]:
    print(row.text)
    print("---")

Name and Location Order Basis for Order
---
NGUYEN, MAI T
City: AUSTIN
County: TRAVIS
Zip Code: 78717


License #: 750076

Complaint # COS20220000853 Date: 7/7/2022

Respondent is assessed an administrative penalty in the amount of $7,500. Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 3 violations; Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 3 violations; Respondent failed to clean and disinfect all wax pots; Respondent failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone.
---
NGUYEN, HUNG VU
City: HOUSTON
County: HARRIS
Zip Code

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – `driver.find_element` or `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [7]:
doc=BeautifulSoup(driver.page_source)

In [8]:
rows=doc.select("tr")

for row in rows:
    try:
        print(row.select_one("span.results_text").text)
    except:
        print("Doesn't have a name")

Doesn't have a name
NGUYEN, MAI T
NGUYEN, HUNG VU 
NGUYEN, MINH HIEN LUONG 
NGUYEN, PHUONG T
NGUYEN, STEVEN MANH 
NGUYEN, LANG THU 
NGUYEN, THANH 
NGUYEN, NAM QUANG 
NGUYEN, JAY TRI 
NGUYEN, HUYEN THANH THI 
NGUYEN, MATTHEW VIET 
NGUYEN, LIEU D
NGUYEN, OANH 
NGUYEN, KIMLIEN 
NGUYEN, JOHNNY DAT 
NGUYEN, NHO THANH 
NGUYEN, KEN 
NGUYEN, MEJ X
NGUYEN, THI THU HUNG 
NGUYEN, LOAN HONG ANH 
NGUYEN, HONG VU 
NGUYEN, LEHUONG 
NGUYEN, HAI THANH 
NGUYEN, CHI 
NGUYEN, THY DINH MINH 
NGUYEN, KATY 
NGUYEN, HIEU V
NGUYEN, ANNA 
NGUYEN, NGA T
NGUYEN, THANH 
NGUYEN, DAI T
NGUYEN, LONG D
NGUYEN, LUCIE HUONG 
NGUYEN, CHINH 
NGUYEN, HUNG VU 
NGUYEN, JIMMY 
NGUYEN, NAM 
NGUYEN, DUC 
NGUYEN, THU THAO THI 
NGUYEN, MINH NHU 
NGUYEN, DUNG VAN 
NGUYEN, TINH 
NGUYEN, HANG NU THANH 
NGUYEN, LAN N
NGUYEN, TIEP HUY 
NGUYEN, DUNG 
NGUYEN, THAI VAN 
NGUYEN, THI THAO UYEN 
NGUYEN, THANH TRONG 
NGUYEN, KHANH V
NGUYEN, DUC VAN 
NGUYEN, THI HOAN CHAU 
NGUYEN, PHUCHUNG 
NGUYEN, KIM 
NGUYEN, HUE VAN 
NGUYEN, HOAI THU T
NGU

## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: If you're using Selenium by itself, you can get the HTML of something by doing `.get_attribute('innerHTML')` – that way it'll look like BeautifulSoup when you print it. It might help you diagnose your issue!*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [36]:
for row in rows:
    try:
        print(row.find_all("td", style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:39%;")[1].text)
    except:
        print("Doesn't have a violation")
    

Doesn't have a violation
Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 3 violations; Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 3 violations; Respondent failed to clean and disinfect all wax pots; Respondent failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone.
Respondent failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondent failed to clean and disinfect multiple use implements after each customer; Respondent failed to keep a record of the date and time of each

## Loop through each result, printing the complaint number

Output should look like this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [10]:
for row in rows:
    try:
        print(row.select("span.results_text")[-2].text)
    except:
        print("Doesn't have a complaint number")

Doesn't have a complaint number
COS20220000853
COS20200013887
COS20210010437
COS20210014779
COS20200015190
COS20200008154
COS20200007790
COS20210010525
COS20210015763
COS20210011311
COS20210003963
COS20190017129
COS20200010645
COS20210012116
COS20210011168
COS20210014198
COS20220008308
COS20210010069
COS20210011374
COS20220001466
COS20210015652
COS20210015023
COS20220001836
COS20210010769
COS20200012226
COS20200006932
COS20210010477
COS20210005808
COS20210008358
COS20210004784
COS20210005027
COS20210009745
COS20210011484
COS20210011721
COS20200006792
COS20200007069
COS20210010530
COS20200007141
COS20200000839
COS20210009714
COS20210005838
COS20200010129
COS20210003441
COS20200009912
COS20210004018
COS20210000935
COS20210003238
COS20200005799
COS20210004209
COS20200012887
COS20190014959
COS20190011954
COS20200016795
COS20200016258
COS20200017028
COS20200016091
COS20210001519
COS20200012575
COS20200007138
COS20200010817
COS20190012445
COS20190010554
COS20200009961
COS20200011967
COS20200

## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [54]:
violations=[]

for row in rows:
    try:
        name=row.select_one("span.results_text").text
        city=row.find_all("span")[2].text
        county=row.find_all("span")[4].text
        zip_code=row.find_all("span")[6].text
        complaint_no=row.select("span.results_text")[-2].text
        license_numbers=row.find_all("span")[8].text
        complaint=row.find_all("td", style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:39%;")[1].text
        
    except:
        pass
    
    violation={
    'name': name,
    'city': city,
    'county': county,
    'zip_code': zip_code,
    'complaint_no': complaint_no,
    'license_numbers': license_numbers,
    'complaint': complaint
    }
    
    violations.append(violation)

    

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [56]:
df=pd.DataFrame(violations)
df.head()

,name,city,county,zip_code,complaint_no,license_numbers,complaint
0,"NGUYEN, KIM NGAN THI",HOUSTON,HARRIS,77084,COS2009004238D,1434311,Respondent interfered with a Department inspec...
1,"NGUYEN, MAI T",AUSTIN,TRAVIS,78717,COS20220000853,750076,"Respondent failed to clean, disinfect, and ste..."
2,"NGUYEN, HUNG VU",HOUSTON,HARRIS,77086,COS20200013887,763912,"Respondent failed to keep floors, walls, ceili..."
3,"NGUYEN, MINH HIEN LUONG",CARROLLTON,DALLAS,75007,COS20210010437,777932,Respondent failed to clean and sanitize whirlp...
4,"NGUYEN, PHUONG T",PLANO,COLLIN,75023,COS20210014779,766293,Respondent failed to keep a record of the date...


In [57]:
df.to_csv("output.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [61]:
pd.read_csv("output.csv").head()

,name,city,county,zip_code,complaint_no,license_numbers,complaint
0,"NGUYEN, KIM NGAN THI",HOUSTON,HARRIS,77084,COS2009004238D,1434311,Respondent interfered with a Department inspec...
1,"NGUYEN, MAI T",AUSTIN,TRAVIS,78717,COS20220000853,750076,"Respondent failed to clean, disinfect, and ste..."
2,"NGUYEN, HUNG VU",HOUSTON,HARRIS,77086,COS20200013887,763912,"Respondent failed to keep floors, walls, ceili..."
3,"NGUYEN, MINH HIEN LUONG",CARROLLTON,DALLAS,75007,COS20210010437,777932,Respondent failed to clean and sanitize whirlp...
4,"NGUYEN, PHUONG T",PLANO,COLLIN,75023,COS20210014779,766293,Respondent failed to keep a record of the date...
